In [1]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/mufa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mufa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mufa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [3]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))

    

In [4]:
#Creating TfIdf vectorizer
ftss=[]
for tweet in filtered_tweets_stemmed:
    ftss.append(' '.join(tweet))
clf = svm.SVC(decision_function_shape='ovo')
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),lowercase=False,max_features=10000,smooth_idf=False,norm=None,max_df=0.75,min_df=5)
X = vectorizer.fit_transform(ftss).toarray()

In [5]:
Y = dataset['class']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)
from sklearn.model_selection import StratifiedKFold, GridSearchCV


In [6]:
print(X)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
print(y_pred)
from sklearn.metrics import classification_report
report = classification_report( y_test, y_pred )
print(report)


[1 1 1 ... 2 1 1]
             precision    recall  f1-score   support

          0       0.75      0.01      0.01       427
          1       0.86      0.98      0.92      5747
          2       0.85      0.57      0.68      1261

avg / total       0.85      0.86      0.82      7435

